# Imbalanced Data Methods
---
Data: http://sci2s.ugr.es/keel/imbalanced.php

In [4]:
import tensorflow as tf
import imblearn

## Naive Downsampling

## Naive Upsampling

## Bagging

## Sigmoid Neural Network

## Sigmoid-weighted sigmoid ensemble